# Problem 1 #

In [2]:
import numpy as np
import pandas as pd

In [9]:
delayed_neutrons_per_thermal_fission = 0.01668
neutrons_per_thermal_fission = 2.43
delayed_group = pd.DataFrame(
    {
        "Group": range(1,7),
        "Half Life": [54.51, 21.84, 6.00, 2.23, 0.496, 0.179],
        "Relative Yield": [0.038, 0.213, 0.188, 0.407, 0.128, 0.026]
    }
)
delayed_group.set_index("Group", inplace=True)
delayed_group

,Half Life,Relative Yield
Group,,
1,54.510,0.038
2,21.840,0.213
3,6.000,0.188
4,2.230,0.407
5,0.496,0.128
6,0.179,0.026


In [10]:
delayed_group["Delayed Neutron Fraction"] =\
    delayed_group["Relative Yield"] * delayed_neutrons_per_thermal_fission / neutrons_per_thermal_fission
delayed_group

,Half Life,Relative Yield,Delayed Neutron Fraction
Group,,,
1,54.510,0.038,0.000261
2,21.840,0.213,0.001462
3,6.000,0.188,0.001290
4,2.230,0.407,0.002794
5,0.496,0.128,0.000879
6,0.179,0.026,0.000178


In [11]:
print "Total delayed neutron fraction: {0}".format(sum(delayed_group["Delayed Neutron Fraction"]))

Total delayed neutron fraction: 0.00686419753086


In [12]:
# verification
(0.01668) / (2.43 + 0.01668)

0.006817401540046103

# Problem 2 #

# Problem 3 #

In [24]:
tau_prompt = 1e-4

In [13]:
delayed_group["Decay Constant"] = np.log(2.) / delayed_group["Half Life"]
delayed_group

,Half Life,Relative Yield,Delayed Neutron Fraction,Decay Constant
Group,,,,
1,54.510,0.038,0.000261,0.012716
2,21.840,0.213,0.001462,0.031738
3,6.000,0.188,0.001290,0.115525
4,2.230,0.407,0.002794,0.310828
5,0.496,0.128,0.000879,1.397474
6,0.179,0.026,0.000178,3.872331


In [22]:
beta = sum(delayed_group["Delayed Neutron Fraction"])

In [25]:
theta = delayed_group["Delayed Neutron Fraction"].dot(delayed_group["Half Life"]) + (1. - beta) * tau_prompt
theta

0.060689942123456783

In [29]:
def k(reactivity):
    return 1. / (1. - reactivity)

def tau(theta, k):
    return theta / np.log(k)

In [34]:
def power(k, t, initial=1000):
    period = tau(theta, k)
    p = initial * np.exp(t / period)
    
    return p

In [36]:
power_plus_twenty_cents = power(k(0.2), 60)
power_plus_twenty_cents

6.431464284016781e+98

In [37]:
power_minus_twenty_cents = power(k(-0.2), 60)
power_minus_twenty_cents

5.2349310466657559e-76